## 2. FEPAA GC 合并历史数据

In [52]:
from datetime import datetime 
import pyspark.sql.functions as F   
import os

# todo 1:指定路径
gc_month_path = 'abfss://test-data@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/test/WZX/01-FEPAA/FEPAA_GC_month.csv'
gc_history_path = 'abfss://test-data@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/test/WZX/01-FEPAA/ods_fepaa_currency_gc_df.csv'
gc_merge_path = 'abfss://test-data@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/test/WZX/01-FEPAA/ods_fepaa_currency_gc_new.csv'


# todo 2:读取df
df1 = spark.read.csv(gc_month_path,header=True)
df2 = spark.read.csv(gc_history_path ,header=True)

# todo 3:合并数据
df2 = df2.drop('etl_load_time').where(F.col('month')!='month')
df_merger = df2.unionAll(df1).dropDuplicates()

# todo 4:添加数据加载时间
formatted_endtime = datetime.now()
etl_load_time = formatted_endtime.strftime("%Y-%m-%d %H:%M:%S")
df = df_merger.withColumn('etl_load_time',F.lit(etl_load_time)) 
# # todo 5:写入文件

df.coalesce(1).write.csv(gc_merge_path,mode='overwrite',header=True)

